In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()

25/09/18 08:05:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df = spark.read.text("/home/iceberg/notebooks/data/WordData.txt")

In [4]:
df.show()

+--------------------+
|               value|
+--------------------+
|This is a Japanes...|
|The team members ...|
|As the years pass...|
|If you don't like...|
|He was disappoint...|
|When he encounter...|
|Situps are a terr...|
|Toddlers feeding ...|
|Edith could decid...|
|Her daily goal wa...|
|Tomorrow will bri...|
|His son quipped t...|
|He wondered why a...|
|If my calculator ...|
|The hummingbird's...|
|He went on a whis...|
|This is the last ...|
|I come from a tri...|
|The delicious aro...|
|Weather is not tr...|
+--------------------+
only showing top 20 rows



In [5]:
df.count()

197

In [6]:
import pyspark.sql.functions as f
from pyspark.sql.functions import lit,col,explode

In [7]:
df2 = df.withColumn("splitedData", f.split("value", " "))
df2.show(truncate=True)

+--------------------+--------------------+
|               value|         splitedData|
+--------------------+--------------------+
|This is a Japanes...|[This, is, a, Jap...|
|The team members ...|[The, team, membe...|
|As the years pass...|[As, the, years, ...|
|If you don't like...|[If, you, don't, ...|
|He was disappoint...|[He, was, disappo...|
|When he encounter...|[When, he, encoun...|
|Situps are a terr...|[Situps, are, a, ...|
|Toddlers feeding ...|[Toddlers, feedin...|
|Edith could decid...|[Edith, could, de...|
|Her daily goal wa...|[Her, daily, goal...|
|Tomorrow will bri...|[Tomorrow, will, ...|
|His son quipped t...|[His, son, quippe...|
|He wondered why a...|[He, wondered, wh...|
|If my calculator ...|[If, my, calculat...|
|The hummingbird's...|[The, hummingbird...|
|He went on a whis...|[He, went, on, a,...|
|This is the last ...|[This, is, the, l...|
|I come from a tri...|[I, come, from, a...|
|The delicious aro...|[The, delicious, ...|
|Weather is not tr...|[Weather, 

In [8]:
df2.select("splitedData").show()

+--------------------+
|         splitedData|
+--------------------+
|[This, is, a, Jap...|
|[The, team, membe...|
|[As, the, years, ...|
|[If, you, don't, ...|
|[He, was, disappo...|
|[When, he, encoun...|
|[Situps, are, a, ...|
|[Toddlers, feedin...|
|[Edith, could, de...|
|[Her, daily, goal...|
|[Tomorrow, will, ...|
|[His, son, quippe...|
|[He, wondered, wh...|
|[If, my, calculat...|
|[The, hummingbird...|
|[He, went, on, a,...|
|[This, is, the, l...|
|[I, come, from, a...|
|[The, delicious, ...|
|[Weather, is, not...|
+--------------------+
only showing top 20 rows



In [9]:
df3 = df2.withColumn("words", explode("splitedData"))
df3.show()

+--------------------+--------------------+--------+
|               value|         splitedData|   words|
+--------------------+--------------------+--------+
|This is a Japanes...|[This, is, a, Jap...|    This|
|This is a Japanes...|[This, is, a, Jap...|      is|
|This is a Japanes...|[This, is, a, Jap...|       a|
|This is a Japanes...|[This, is, a, Jap...|Japanese|
|This is a Japanes...|[This, is, a, Jap...|    doll|
|The team members ...|[The, team, membe...|     The|
|The team members ...|[The, team, membe...|    team|
|The team members ...|[The, team, membe...| members|
|The team members ...|[The, team, membe...|    were|
|The team members ...|[The, team, membe...|    hard|
|The team members ...|[The, team, membe...|      to|
|The team members ...|[The, team, membe...|    tell|
|The team members ...|[The, team, membe...|   apart|
|The team members ...|[The, team, membe...|   since|
|The team members ...|[The, team, membe...|    they|
|The team members ...|[The, team, membe...|   

In [10]:
wordsDF = df3.select("words")
wordsDF.show()

+--------+
|   words|
+--------+
|    This|
|      is|
|       a|
|Japanese|
|    doll|
|     The|
|    team|
| members|
|    were|
|    hard|
|      to|
|    tell|
|   apart|
|   since|
|    they|
|     all|
|    wore|
|   their|
|    hair|
|      in|
+--------+
only showing top 20 rows



In [11]:
wordCount = wordsDF.groupBy("words").count()
wordCount.show()

+-----------+-----+
|      words|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



In [12]:
driver = "com.mysql.cj.jdbc.Driver"
url = "jdbc:mysql://karthick1808.c7i86mqiit2n.ap-south-1.rds.amazonaws.com:3306/spark"
table = "wordcount"
user = "karthick1808"
password = "Welcome!12345"

In [ ]:
wordCount.write.format("jdbc").option("driver", driver).option("url",url).option("dbtable", table).option("mode", "append").option("user",user).option("password", password).save()

[Stage 13:>                                                         (0 + 1) / 1]